In [ ]:
%matplotlib notebook

In [ ]:
%matplotlib inline

In [ ]:
from adler.dataclasses.AdlerPlanetoid import AdlerPlanetoid
from adler.dataclasses.AdlerData import AdlerData
from adler.science.PhaseCurve import PhaseCurve
from adler.science.Colour import col_obs_ref
from adler.utilities.plotting_utilities import plot_errorbar
from adler.utilities.science_utilities import apparition_gap_finder, get_df_obs_filt

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import astropy.units as u
from scipy.interpolate import splrep, BSpline, CubicSpline
from astropy.modeling.fitting import SLSQPLSQFitter
from astropy.time import Time
from astroquery.jplhorizons import Horizons

In [ ]:
# ssObjectId of object to analyse
# ssoid = "8268570668335894776" # NEO
ssoid = "6098332225018"  # good MBA test object

In [ ]:
# fname = "../tests/data/testing_database.db"
fname = "/Users/jrobinson/lsst-adler/notebooks/gen_test_data/adler_demo_testing_database.db"
planetoid = AdlerPlanetoid.construct_from_SQL(ssoid, sql_filename=fname)

In [ ]:
# known issue with mpcDesignation
# https://dp0-3.lsst.io/data-products-dp0-3/data-simulation-dp0-3.html#known-issues
planetoid.MPCORB.mpcDesignation

In [ ]:
planetoid.MPCORB.fullDesignation

In [ ]:
mpc_des = planetoid.MPCORB.fullDesignation
mpc_des = mpc_des.split("2011 ")[-1]
planetoid.MPCORB.fullDesignation = mpc_des
mpc_des

In [ ]:
# check orbit
e = planetoid.MPCORB.e
incl = planetoid.MPCORB.incl
q = planetoid.MPCORB.q
a = q / (1.0 - e)
Q = a * (1.0 + e)
print(a, e, incl)

In [ ]:
dir(planetoid)

In [ ]:
planetoid.SSObject

In [ ]:
fig = plt.figure()
gs = gridspec.GridSpec(1, 1)
ax1 = plt.subplot(gs[0, 0])

adler_cols = AdlerData(ssoid, planetoid.filter_list)

for filt in ["g", "r"]:
    # get observations and define a phase curve model to fit

    sso = planetoid.SSObject_in_filter(filt)
    obs = planetoid.observations_in_filter(filt)

    H = sso.H
    G12 = sso.G12
    pc = PhaseCurve(H=H * u.mag, phase_parameter_1=G12, model_name="HG12_Pen16")

    # H = sso.H
    # S = 0.2
    # pc = PhaseCurve(H=H * u.mag, phase_parameter_1=S* (u.mag/u.deg), model_name="LinearPhaseFunc")

    # H = sso.H
    # G1 = 0.2
    # G2 = 0.2
    # pc = PhaseCurve(H=H * u.mag, phase_parameter_1=G1, phase_parameter_2=G2, model_name="HG1G2")

    alpha = np.linspace(0, np.amax(obs.phaseAngle)) * u.deg
    # red_mag = pc.ReducedMag(alpha)
    # pc.model_function.G12.fixed = True
    # pc.FixParam("S")
    # pc.FixParam("phase_parameter_1")
    # pc.model_function.H.fixed = True
    # pc.model_function.G1.fixed = True
    # pc.model_function.G2.fixed = True

    pc_fit = pc.FitModel(
        np.array(getattr(obs, "phaseAngle")) * u.deg,
        np.array(getattr(obs, "reduced_mag")) * u.mag,
        #     np.array(getattr(obs, "magErr")) * u.mag,
        #     fitter = SLSQPLSQFitter()
    )
    pc = pc.InitModelSbpy(pc_fit)
    red_mag = pc.ReducedMag(alpha)

    adler_cols.populate_phase_parameters(filt, **pc.ReturnModelDict())

    # add this phase curve to the figure
    fig = plot_errorbar(planetoid, filt_list=[filt], fig=fig)
    ax1 = fig.axes[0]
    ax1.plot(
        alpha.value,
        pc.ReducedMag(alpha).value,
        label="{}: H={:.2f}, G12={:.2f}".format(filt, pc.H, pc.phase_parameter_1),
    )
    ax1.legend()

ax1.invert_yaxis()

plt.show()

In [ ]:
adler_cols.get_phase_parameters_in_filter("r", "HG12_Pen16").__dict__

In [ ]:
adler_cols.get_phase_parameters_in_filter("g", "HG12_Pen16").__dict__

In [ ]:
planetoid.SSObject_in_filter("g")

In [ ]:
# # add this phase curve to the figure
# fig = plot_errorbar(planetoid, filt_list=[filt])
# ax1 = fig.axes[0]
# ax1.plot(alpha.value, pc.ReducedMag(alpha).value, label="{} {}".format(filt, pc.model_name))
# ax1.legend()
# plt.show()

In [ ]:
# calculate data - model residuals
res = pc.ModelResiduals(obs.phaseAngle * u.deg, obs.reduced_mag * u.mag).value
res

In [ ]:
# calculate the absolute mag
abs_mag = pc.AbsMag(obs.phaseAngle * u.deg, obs.reduced_mag * u.mag).value
abs_mag

In [ ]:
# x_plot = "phaseAngle"
x_plot = "midPointMjdTai"
x = getattr(obs, x_plot)
# y = res
y = abs_mag
yerr = obs.magErr

sort_mask = np.argsort(x)
x = x[sort_mask]
y = y[sort_mask]

tck1 = splrep(x, y, s=0)
tck2 = splrep(x, y, s=len(x))
tck3 = splrep(x, y, w=1.0 / yerr)

In [ ]:
tck3

In [ ]:
len(tck1[0])

In [ ]:
# plot the data - model residuals
# x_plot = "phaseAngle"
x_plot = "midPointMjdTai"

x = getattr(obs, x_plot)
yerr = obs.magErr

fig = plt.figure()
gs = gridspec.GridSpec(1, 1)
ax1 = plt.subplot(gs[0, 0])

# ax1.errorbar(x, res, yerr, fmt="o")
ax1.errorbar(x, abs_mag, yerr, fmt="o")

# ax1.axhline(0, c="k")
ax1.axhline(pc.H.value, c="k")

xnew = np.linspace(np.amin(x), np.amax(x), 1000)
# ax1.plot(xnew, BSpline(*tck1)(xnew), '-', label='s=0')
ax1.plot(xnew, BSpline(*tck2)(xnew), "-", label=f"s={len(x)}")
ax1.plot(xnew, BSpline(*tck3)(xnew), "-", label="weighted smooth")

ax1.invert_yaxis()
ax1.set_xlabel(x_plot)
ax1.set_ylabel("data - model")
ax1.legend()

plt.show()

In [ ]:
# # use a running mean?

# def movingaverage(interval, window_size):
#     window = np.ones(int(window_size))/float(window_size)
#     return np.convolve(interval, window, 'same')

In [ ]:
# # plot the data - model residuals
# x_plot = "phaseAngle"

# x = getattr(obs, x_plot)
# yerr = obs.magErr

# fig = plt.figure()
# gs = gridspec.GridSpec(1, 1)
# ax1 = plt.subplot(gs[0, 0])

# # ax1.errorbar(x, res, yerr, fmt="o")
# ax1.errorbar(x, abs_mag, yerr, fmt="o")

# # ax1.axhline(0, c="k")
# ax1.axhline(pc.H.value, c="k")

# xnew = np.linspace(np.amin(x),np.amax(x),1000)
# # ax1.plot(xnew, BSpline(*tck1)(xnew), '-', label='s=0')
# ax1.plot(xnew, BSpline(*tck2)(xnew), '-', label=f's={len(x)}')
# ax1.plot(xnew, BSpline(*tck3)(xnew), '-', label='weighted smooth')
# ax1.scatter(x, BSpline(*tck3)(x), c = "C2")


# # x_plot = "phaseAngle"
# # x = getattr(obs, x_plot)
# # # y = res
# # y = abs_mag
# # sort_mask = np.argsort(x)
# # x = x[sort_mask]
# # y = y[sort_mask]
# # y_av = movingaverage(y, 5)
# # ax1.plot(x, y_av,label = "moving avg")


# ax1.invert_yaxis()
# ax1.set_xlabel(x_plot)
# ax1.set_ylabel("data - model")
# ax1.legend()

# plt.show()

In [ ]:
# generate the spline points at all points of a given filter set
# find the difference at these points
# do we consider differences where there is no data?

In [ ]:
# plot the data - model residuals
# x_plot = "phaseAngle"
x_plot = "midPointMjdTai"

fig = plt.figure()
gs = gridspec.GridSpec(1, 1)
ax1 = plt.subplot(gs[0, 0])

# # get min,max phase angle
# phase_min = []
# phase_max = []
# for filt in ["g","r"]:
#     obs = planetoid.observations_in_filter(filt)
#     phase_min.append(np.amin(obs.phaseAngle))
#     phase_max.append(np.amax(obs.phaseAngle))
# # phase_min = np.amin(np.array(phase_min))
# # phase_max = np.amax(np.array(phase_max))

# # min and max that encloses both data sets
# phase_min = np.amax(np.array(phase_min))
# phase_max = np.amin(np.array(phase_max))
# print(phase_min,phase_max)

# get min,max time
time_min = []
time_max = []
for filt in ["g", "r"]:
    obs = planetoid.observations_in_filter(filt)
    time_min.append(np.amin(obs.midPointMjdTai))
    time_max.append(np.amax(obs.midPointMjdTai))
# min and max that encloses both data sets
time_min = np.amax(np.array(time_min))
time_max = np.amin(np.array(time_max))
print(time_min, time_max)

splines = []
xdata = np.array([])
for filt in ["g", "r"]:
    # get observations in filter
    obs = planetoid.observations_in_filter(filt)
    # get the stored AdlerData parameters for this filter
    ad = adler_cols.get_phase_parameters_in_filter(filt, "HG12_Pen16")
    # make the PhaseCurve object from AdlerData
    pc = PhaseCurve().InitModelDict(ad.__dict__)
    print(pc.__dict__)

    # calculate the model absolute magnitude
    abs_mag = pc.AbsMag(obs.phaseAngle * u.deg, obs.reduced_mag * u.mag).value

    x = getattr(obs, x_plot)
    xdata = np.concatenate([xdata, x])
    yerr = obs.magErr

    # ax1.errorbar(x, res, yerr, fmt="o")
    ax1.errorbar(x, abs_mag, yerr, fmt="o", label=filt)

    # ax1.axhline(0, c="k")
    ax1.axhline(pc.H.value, c="k", label="H_{} = {:.2f} mag".format(filt, pc.H.value))

    y = abs_mag
    sort_mask = np.argsort(x)
    x = x[sort_mask]
    y = y[sort_mask]
    tck = splrep(x, y, w=1.0 / yerr)
    splines.append(tck)
    #     xnew = np.linspace(np.amin(x),np.amax(x),1000)
    #     xnew = np.linspace(phase_min,phase_max,1000)
    xnew = np.linspace(time_min, time_max, 1000)
    ax1.plot(xnew, BSpline(*tck)(xnew), "-", label="weighted smooth")

#     break

ax1.invert_yaxis()
ax1.set_xlabel(x_plot)
ax1.set_ylabel("abs_mag")
ax1.legend()

plt.show()

In [ ]:
fig = plt.figure()
gs = gridspec.GridSpec(1, 1)
ax1 = plt.subplot(gs[0, 0])

# xnew = np.linspace(phase_min,phase_max,1000)
xnew = np.linspace(time_min, time_max, 1000)
spline_diff = BSpline(*splines[0])(xnew) - BSpline(*splines[1])(xnew)
ax1.plot(xnew, spline_diff, "-", label="weighted smooth", c="k", zorder=1)

phase = np.array([])
spline_diff = np.array([])
for i, filt in enumerate(["g", "r"]):
    obs = planetoid.observations_in_filter(filt)
    #     x = getattr(obs, "phaseAngle")
    x = getattr(obs, "midPointMjdTai")
    yerr = obs.magErr
    #     xdata_mask = (x>=phase_min) & (x<=phase_max)
    xdata_mask = (x >= time_min) & (x <= time_max)
    _spline_diff = BSpline(*splines[0])(x[xdata_mask]) - BSpline(*splines[1])(x[xdata_mask])
    #     ax1.scatter(x[xdata_mask], _spline_diff,
    #                 label = filt, facecolor = "none", edgecolor = "C{}".format(i))
    ax1.errorbar(x[xdata_mask], _spline_diff, yerr[xdata_mask], fmt="o", label=filt)

    phase = np.concatenate([phase, x])
    spline_diff = np.concatenate([spline_diff, _spline_diff])

ad_g = adler_cols.get_phase_parameters_in_filter("g", "HG12_Pen16")
ad_r = adler_cols.get_phase_parameters_in_filter("r", "HG12_Pen16")
ax1.axhline(ad_g.H.value - ad_r.H.value, label="H_g - H_r")

spline_diff_med = np.median(spline_diff)
spline_diff_std = np.std(spline_diff)
ax1.axhline(spline_diff_med, label="median spline diff", c="k")
for i in [3, 2, 1]:
    ax1.axhspan(
        spline_diff_med - (i * spline_diff_std),
        spline_diff_med + (i * spline_diff_std),
        zorder=0,
        color="k",
        alpha=0.15,
    )

ax1.legend()
ax1.set_xlabel("phaseAngle")
ax1.set_ylabel("colour")

plt.show()

In [ ]:
# account for photometric uncertainty!

In [ ]:
# should we fit the spline in phase angle or time space?
# if time, should we consider apparitions separately?

In [ ]:
# find apparitions, use time/solar elongation?

In [ ]:
df_obs = pd.DataFrame(obs.__dict__)

In [ ]:
Time(np.amin(df_obs["midPointMjdTai"]), format="mjd").iso

In [ ]:
Time(np.amax(df_obs["midPointMjdTai"]), format="mjd").iso

In [ ]:
R = df_obs["heliocentricDist"]
Delta = df_obs["topocentricDist"]
alpha = np.radians(df_obs["phaseAngle"])

In [ ]:
# eps1 = (R * np.tan(alpha))
# eps2 = (Delta/np.cos(alpha)) - R
# df_obs["elong"] = np.degrees(np.arctan(eps1/eps2))

In [ ]:
# R_E = np.sqrt((R*R) + (Delta*Delta) - (2.0*R*Delta*np.cos(alpha)))
# df_obs["elong"] = np.degrees(np.arcsin((R/R_E) * np.sin(alpha)))

R_E = np.sqrt((R * R) + (Delta * Delta) - (2.0 * R * Delta * np.cos(alpha)))
df_obs["elong"] = np.degrees(np.arccos(((R_E * R_E) + (Delta * Delta) - (R * R)) / (2.0 * R_E * Delta)))

In [ ]:
# _R = np.sqrt((R_E*R_E) + (Delta*Delta) - (2.0*R_E*Delta*np.cos(np.radians(df_obs["elong"]))))

In [ ]:
df_obs = df_obs.sort_values("midPointMjdTai")
df_obs

In [ ]:
fig = plt.figure()
gs = gridspec.GridSpec(1, 1)
ax1 = plt.subplot(gs[0, 0])

ax1.hist(np.diff(df_obs["midPointMjdTai"]), bins="auto")

plt.show()

In [ ]:
_R = np.sqrt(
    (df_obs["heliocentricX"] ** 2.0) + (df_obs["heliocentricY"] ** 2.0) + (df_obs["heliocentricZ"] ** 2.0)
)
_Delta = np.sqrt(
    (df_obs["topocentricX"] ** 2.0) + (df_obs["topocentricY"] ** 2.0) + (df_obs["topocentricZ"] ** 2.0)
)

In [ ]:
fig = plt.figure()
gs = gridspec.GridSpec(1, 1)
ax1 = plt.subplot(gs[0, 0])

ax1.scatter(df_obs["midPointMjdTai"], df_obs["phaseAngle"])

ax1.set_xlabel("mjd")
ax1.set_ylabel("phaseAngle")

plt.show()

In [ ]:
fig = plt.figure()
gs = gridspec.GridSpec(1, 1)
ax1 = plt.subplot(gs[0, 0])

ax1.scatter(df_obs["midPointMjdTai"], df_obs["heliocentricDist"])
ax1.axhline(q, label="perihelion", ls=":")
ax1.axhline(Q, label="aphelion", ls="--")

ax1.legend()
ax1.set_xlabel("mjd")
ax1.set_ylabel("AU")
plt.show()

In [ ]:
fig = plt.figure()
gs = gridspec.GridSpec(1, 1)
ax1 = plt.subplot(gs[0, 0])

ax1.scatter(df_obs["midPointMjdTai"], df_obs["elong"])

plt.show()

In [ ]:
fig = plt.figure()
gs = gridspec.GridSpec(1, 1)
ax1 = plt.subplot(gs[0, 0])

ax1.scatter(df_obs["phaseAngle"], df_obs["elong"], c=df_obs["midPointMjdTai"])
ax1.axhline(90)
plt.show()

In [ ]:
np.amin(df_obs["elong"])

In [ ]:
# astroquery the object
# target = "2007 YP19"
# target = "2014 QL433"
target = planetoid.MPCORB.fullDesignation

site = "X05"
times = {"start": "2023-10-01 04:00", "stop": "2033-10-01 04:00", "step": "1day"}  # dates to query
obj = Horizons(id=target, location=site, epochs=times)
eph = obj.ephemerides()
df_eph = eph.to_pandas()

In [ ]:
target

In [ ]:
obj = Horizons(id=target, epochs=times, location="399")
vec = obj.vectors()
df_vec_earth = vec.to_pandas()

obj = Horizons(id=target, epochs=times, location="@10")
vec = obj.vectors()
df_vec_sun = vec.to_pandas()

In [ ]:
df_vec_earth = df_vec_earth.rename({"x": "topocentricX", "y": "topocentricY", "z": "topocentricZ"}, axis=1)
df_vec_sun = df_vec_sun.rename({"x": "heliocentricX", "y": "heliocentricY", "z": "heliocentricZ"}, axis=1)

In [ ]:
df_vec = df_vec_earth[
    ["targetname", "datetime_jd", "datetime_str", "topocentricX", "topocentricY", "topocentricZ"]
].merge(
    df_vec_sun[
        ["targetname", "datetime_jd", "datetime_str", "heliocentricX", "heliocentricY", "heliocentricZ"]
    ],
    on=["targetname", "datetime_jd", "datetime_str"],
)

In [ ]:
df_eph["datetime_mjd"] = Time(df_eph["datetime_jd"], format="jd").mjd

In [ ]:
df_vec["datetime_mjd"] = Time(df_vec["datetime_jd"], format="jd").mjd

In [ ]:
eph["datetime_jd"].unit

In [ ]:
df_eph.columns

In [ ]:
# plot only night time data points, replace day time with nans (will leave blank in plot)
# night_mask = (~np.isin(df_eph["solar_presence"],["*","N","C"])) & (df_eph["EL"]>0)
night_mask = (~np.isin(df_eph["solar_presence"], ["*", "N", "C"])) & (df_eph["EL"] > 0)

In [ ]:
df_eph[["solar_presence", "EL"]]

In [ ]:
fig = plt.figure()
gs = gridspec.GridSpec(1, 1)
ax1 = plt.subplot(gs[0, 0])

ax1.scatter(
    df_obs["midPointMjdTai"],
    df_obs["heliocentricDist"],
    edgecolor="C0",
    facecolor="none",
    label="heliocentricDist",
)
# ax1.scatter(df_obs["midPointMjdTai"],_R, marker = "x", c = "C0")

ax1.scatter(
    df_obs["midPointMjdTai"],
    df_obs["topocentricDist"],
    edgecolor="C1",
    facecolor="none",
    label="topocentricDist",
)
# ax1.scatter(df_obs["midPointMjdTai"],_Delta, marker = "x", c = "C1")

ax1.scatter(df_obs["midPointMjdTai"], R_E, edgecolor="C2", facecolor="none", label="heliocentricDist_Earth")
# ax1.scatter(df_obs["midPointMjdTai"],_R_E, marker = "x", c = "C2")

ax1.plot(df_eph["datetime_mjd"], df_eph["r"], c="C0")
ax1.plot(df_eph["datetime_mjd"], df_eph["delta"], c="C1")

ax1.legend()
ax1.set_xlabel("mjd")
ax1.set_ylabel("AU")
plt.show()

In [ ]:
fig = plt.figure()
gs = gridspec.GridSpec(1, 1)
ax1 = plt.subplot(gs[0, 0])

ax1.scatter(
    df_obs["midPointMjdTai"], df_obs["heliocentricX"], edgecolor="C0", facecolor="none", label="heliocentricX"
)
ax1.scatter(
    df_obs["midPointMjdTai"], df_obs["heliocentricY"], edgecolor="C1", facecolor="none", label="heliocentricY"
)
ax1.scatter(
    df_obs["midPointMjdTai"], df_obs["heliocentricZ"], edgecolor="C2", facecolor="none", label="heliocentricZ"
)

ax1.plot(df_vec["datetime_mjd"], df_vec["heliocentricX"], c="C0")
ax1.plot(df_vec["datetime_mjd"], df_vec["heliocentricY"], c="C1")
ax1.plot(df_vec["datetime_mjd"], df_vec["heliocentricZ"], c="C2")

ax1.legend()
ax1.set_xlabel("mjd")
ax1.set_ylabel("AU")
plt.show()

In [ ]:
fig = plt.figure()
gs = gridspec.GridSpec(1, 1)
ax1 = plt.subplot(gs[0, 0])

ax1.scatter(
    df_obs["midPointMjdTai"], df_obs["topocentricX"], edgecolor="C0", facecolor="none", label="topocentricX"
)
ax1.scatter(
    df_obs["midPointMjdTai"], df_obs["topocentricY"], edgecolor="C1", facecolor="none", label="topocentricY"
)
ax1.scatter(
    df_obs["midPointMjdTai"], df_obs["topocentricZ"], edgecolor="C2", facecolor="none", label="topocentricZ"
)

ax1.plot(df_vec["datetime_mjd"], df_vec["topocentricX"], c="C0")
ax1.plot(df_vec["datetime_mjd"], df_vec["topocentricY"], c="C1")
ax1.plot(df_vec["datetime_mjd"], df_vec["topocentricZ"], c="C2")

ax1.legend()
ax1.set_xlabel("mjd")
ax1.set_ylabel("AU")
plt.show()

In [ ]:
x_plot = "midPointMjdTai"
y_plot = "elong"
df_plot = df_obs

fig = plt.figure()
gs = gridspec.GridSpec(1, 1)
ax1 = plt.subplot(gs[0, 0])

ax1.scatter(df_plot[x_plot], df_plot[y_plot], label="DP0.3")
ax1.plot(df_eph["datetime_mjd"], df_eph["elong"], c="r", label="JPL")
ax1.legend()

plt.show()

In [ ]:
x_plot = "midPointMjdTai"
y_plot = "phaseAngle"
df_plot = df_obs

fig = plt.figure()
gs = gridspec.GridSpec(1, 1)
ax1 = plt.subplot(gs[0, 0])

ax1.scatter(df_plot[x_plot], df_plot[y_plot], label="DP0.3", zorder=3)
ax1.plot(df_eph["datetime_mjd"], df_eph["alpha"], c="k", label="JPL", zorder=0)
ax1.scatter(df_eph[night_mask]["datetime_mjd"], df_eph[night_mask]["alpha"], s=10)
ax1.legend()

plt.show()

In [ ]:
df_eph[df_eph["alpha"] > 120][["datetime_str", "alpha", "EL"]]

In [ ]:
Time(np.array(df_obs[df_obs["phaseAngle"] > 120][["midPointMjdTai"]]), format="mjd").iso

In [ ]:
x_plot = "alpha"
y_plot = "elong"
c_plot = "datetime_jd"
df_plot = df_eph[night_mask]

fig = plt.figure()
gs = gridspec.GridSpec(1, 1)
ax1 = plt.subplot(gs[0, 0])

ax1.scatter(df_plot[x_plot], df_plot[y_plot], c=df_plot[c_plot])
ax1.axhline(90)

ax1.scatter(df_obs["phaseAngle"], df_obs["elong"], c="r", marker="x")

plt.show()

In [ ]:
x_plot = "r"
y_plot = "delta"
c_plot = "datetime_jd"
df_plot = df_eph[night_mask]

fig = plt.figure()
gs = gridspec.GridSpec(1, 1)
ax1 = plt.subplot(gs[0, 0])

ax1.scatter(df_plot[x_plot], df_plot[y_plot], c=df_plot[c_plot])

ax1.scatter(df_obs["heliocentricDist"], df_obs["topocentricDist"], c="r", marker="x")

plt.show()

In [ ]:
x_plot = "r"
y_plot = "alpha"
c_plot = "datetime_jd"
df_plot = df_eph[night_mask]

fig = plt.figure()
gs = gridspec.GridSpec(1, 1)
ax1 = plt.subplot(gs[0, 0])

ax1.scatter(df_plot[x_plot], df_plot[y_plot], c=df_plot[c_plot])

ax1.scatter(df_obs["heliocentricDist"], df_obs["phaseAngle"], c="r", marker="x")

plt.show()

In [ ]:
x_plot = "delta"
y_plot = "alpha"
c_plot = "datetime_jd"
df_plot = df_eph[night_mask]

fig = plt.figure()
gs = gridspec.GridSpec(1, 1)
ax1 = plt.subplot(gs[0, 0])

ax1.scatter(df_plot[x_plot], df_plot[y_plot], c=df_plot[c_plot])

ax1.scatter(df_obs["topocentricDist"], df_obs["phaseAngle"], c="r", marker="x")

plt.show()

In [ ]:
# look up rsp notebooks to calculate x, y, z positions and to check orbits and angles

In [ ]:
df_obs_all = pd.DataFrame()
for filt in ["r", "g"]:
    obs = planetoid.observations_in_filter(filt)
    _df_obs = pd.DataFrame(obs.__dict__)
    df_obs_all = pd.concat([df_obs_all, _df_obs])
df_obs_all = df_obs_all.sort_values("midPointMjdTai")

t_app = apparition_gap_finder(np.array(df_obs_all["midPointMjdTai"]))

In [ ]:
df_obs_all

In [ ]:
x_plot = "heliocentricX"
y_plot = "heliocentricY"
z_plot = "heliocentricZ"
df_plot = df_obs_all

fig = plt.figure()
ax1 = fig.add_subplot(projection="3d")

ax1.scatter(df_plot[x_plot], df_plot[y_plot], df_plot[z_plot])

ax1.set_aspect("equal")

plt.show()

In [ ]:
x_plot = "topocentricX"
y_plot = "topocentricY"
z_plot = "topocentricZ"
df_plot = df_obs_all

fig = plt.figure()
ax1 = fig.add_subplot(projection="3d")

ax1.scatter(df_plot[x_plot], df_plot[y_plot], df_plot[z_plot])

ax1.set_aspect("equal")

plt.show()

In [ ]:
x_plot = "heliocentricX"
y_plot = "heliocentricY"
c_plot = "midPointMjdTai"
df_plot = df_obs_all

fig = plt.figure()
gs = gridspec.GridSpec(1, 1)
ax1 = plt.subplot(gs[0, 0])

s1 = ax1.scatter(df_plot[x_plot], df_plot[y_plot], c=df_plot[c_plot])
cbar1 = plt.colorbar(s1)

ax1.scatter(0, 0, marker="+", c="k")
circle1 = plt.Circle((0, 0), 1.0, edgecolor="r", facecolor="none", label="1au")
ax1.add_patch(circle1)

mask = df_plot["phaseAngle"] > 120
_df_plot = df_plot[mask]
ax1.scatter(_df_plot[x_plot], _df_plot[y_plot], facecolor="none", edgecolor="r")

ax1.set_aspect("equal")
ax1.legend()

cbar1.set_label(c_plot)
ax1.set_xlabel(x_plot)
ax1.set_ylabel(y_plot)

plt.show()

In [ ]:
# use simple time diff for now

In [ ]:
x = np.array(df_obs_all["midPointMjdTai"])
x[0], x[-1]

In [ ]:
t_app

In [ ]:
x_plot = "midPointMjdTai"
y_plot = "reduced_mag"
df_plot = df_obs_all

fig = plt.figure()
gs = gridspec.GridSpec(1, 1)
ax1 = plt.subplot(gs[0, 0])

# for filt in ["r","g"]:
for filt in ["g", "r"]:
    _df_plot = df_plot[df_plot["filter_name"] == filt]
    ax1.scatter(_df_plot[x_plot], _df_plot[y_plot], label=filt)

for x in t_app:
    ax1.axvline(x, c="k")

ax1.set_xlabel(x_plot)
ax1.set_ylabel(y_plot)
ax1.legend()
ax1.invert_yaxis()

plt.show()

In [ ]:
x_plot = "midPointMjdTai"

fig = plt.figure()
gs = gridspec.GridSpec(1, 1)
ax1 = plt.subplot(gs[0, 0])

for i in range(len(t_app) - 1):
    if i != 3:
        continue

    time_min = t_app[i]
    time_max = t_app[i + 1]
    print(time_min, time_max)

    _df_obs_all = df_obs_all[
        (df_obs_all["midPointMjdTai"] >= time_min) & (df_obs_all["midPointMjdTai"] < time_max)
    ]
    print(i, len(_df_obs_all))
    _time_max = np.amax(_df_obs_all["midPointMjdTai"])

    splines = []
    xdata = np.array([])
    for filt in ["g", "r"]:
        # get observations in filter
        obs = planetoid.observations_in_filter(filt)
        # get the stored AdlerData parameters for this filter
        ad = adler_cols.get_phase_parameters_in_filter(filt, "HG12_Pen16")
        # make the PhaseCurve object from AdlerData
        pc = PhaseCurve().InitModelDict(ad.__dict__)
        #         print(pc.__dict__)

        # calculate the model absolute magnitude
        abs_mag = pc.AbsMag(obs.phaseAngle * u.deg, obs.reduced_mag * u.mag).value

        x = getattr(obs, x_plot)
        xdata = np.concatenate([xdata, x])

        y = abs_mag
        sort_mask = np.argsort(x)
        x = x[sort_mask]
        y = y[sort_mask]
        yerr = obs.magErr[sort_mask]
        #         x_mask = (x>=time_min) & (x<time_max)
        x_mask = (x >= time_min) & (x <= _time_max)
        x = x[x_mask]
        y = y[x_mask]
        yerr = yerr[x_mask]
        #         print(len(x))

        ax1.errorbar(x, y, yerr, fmt="o", label=filt)

        # ax1.axhline(0, c="k")
        ax1.axhline(pc.H.value, c="k", label="H_{} = {:.2f} mag".format(filt, pc.H.value))

        if len(x) < 5:
            continue

        tck = splrep(x, y, w=1.0 / yerr)
        splines.append(tck)

        #         xnew = np.linspace(time_min,time_max,1000)
        xnew = np.linspace(time_min, _time_max, 1000)
        ax1.plot(xnew, BSpline(*tck)(xnew), "-", label="weighted smooth", c="k", zorder=3)

# ax1.legend()
ax1.set_ylim(pc.H.value - 1, pc.H.value + 1)

plt.show()

In [ ]:
x_plot = "midPointMjdTai"

fig = plt.figure()
gs = gridspec.GridSpec(1, 1)
ax1 = plt.subplot(gs[0, 0])

for i in range(len(t_app) - 1):
    if i != 3:
        continue

    time_min = t_app[i]
    time_max = t_app[i + 1]
    print(time_min, time_max)

    _df_obs_all = df_obs_all[
        (df_obs_all["midPointMjdTai"] >= time_min) & (df_obs_all["midPointMjdTai"] < time_max)
    ]
    print(i, len(_df_obs_all))
    _time_max = np.amax(_df_obs_all["midPointMjdTai"])

    splines = []
    xdata = np.array([])
    for filt in ["g", "r"]:
        # get observations in filter
        obs = planetoid.observations_in_filter(filt)
        # get the stored AdlerData parameters for this filter
        ad = adler_cols.get_phase_parameters_in_filter(filt, "HG12_Pen16")
        # make the PhaseCurve object from AdlerData
        pc = PhaseCurve().InitModelDict(ad.__dict__)
        #         print(pc.__dict__)

        # calculate the model absolute magnitude
        abs_mag = pc.AbsMag(obs.phaseAngle * u.deg, obs.reduced_mag * u.mag).value

        x = getattr(obs, x_plot)
        xdata = np.concatenate([xdata, x])

        y = abs_mag
        sort_mask = np.argsort(x)
        x = x[sort_mask]
        y = y[sort_mask]
        yerr = obs.magErr[sort_mask]
        #         x_mask = (x>=time_min) & (x<time_max)
        x_mask = (x >= time_min) & (x <= _time_max)
        x = x[x_mask]
        y = y[x_mask]
        yerr = yerr[x_mask]
        #         print(len(x))

        if len(x) < 5:
            continue

        tck = splrep(x, y, w=1.0 / yerr)
        splines.append(tck)

    if len(splines) < 2:
        continue

    #     xnew = np.linspace(time_min,time_max,1000)
    xnew = np.linspace(time_min, _time_max, 1000)
    spline_diff = BSpline(*splines[0])(xnew) - BSpline(*splines[1])(xnew)
    ax1.plot(xnew, spline_diff, "-", label="weighted smooth", c="k", zorder=1)

    phase = np.array([])
    spline_diff = np.array([])
    for i, filt in enumerate(["g", "r"]):
        obs = planetoid.observations_in_filter(filt)
        x = getattr(obs, "midPointMjdTai")
        yerr = obs.magErr
        #         xdata_mask = (x>=time_min) & (x<time_max)
        xdata_mask = (x >= time_min) & (x <= _time_max)
        _spline_diff = BSpline(*splines[0])(x[xdata_mask]) - BSpline(*splines[1])(x[xdata_mask])
        ax1.errorbar(x[xdata_mask], _spline_diff, yerr[xdata_mask], fmt="o", label=filt)

        phase = np.concatenate([phase, x])
        spline_diff = np.concatenate([spline_diff, _spline_diff])

    ad_g = adler_cols.get_phase_parameters_in_filter("g", "HG12_Pen16")
    ad_r = adler_cols.get_phase_parameters_in_filter("r", "HG12_Pen16")
    ax1.axhline(ad_g.H.value - ad_r.H.value, label="H_g - H_r")

#     spline_diff_med = np.median(spline_diff)
#     spline_diff_std = np.std(spline_diff)
#     ax1.axhline(spline_diff_med,label = "median spline diff", c = "k")
#     for i in [3,2,1]:
#         ax1.axhspan(spline_diff_med - (i*spline_diff_std), spline_diff_med + (i*spline_diff_std),
#                    zorder = 0, color = "k", alpha = 0.15)

# ax1.legend()

plt.show()

In [ ]:
x_plot = "midPointMjdTai"
y_plot = "reduced_mag"

fig = plt.figure()
gs = gridspec.GridSpec(1, 1)
ax1 = plt.subplot(gs[0, 0])

for i in range(len(t_app) - 1):
    if i != 3:
        continue

    time_min = t_app[i]
    time_max = t_app[i + 1]
    print(time_min, time_max)

    _df_obs_all = df_obs_all[
        (df_obs_all["midPointMjdTai"] >= time_min) & (df_obs_all["midPointMjdTai"] < time_max)
    ]
    print(i, len(_df_obs_all))
    _time_max = np.amax(_df_obs_all["midPointMjdTai"])

    for filt in ["g", "r"]:
        # get observations in filter
        obs = planetoid.observations_in_filter(filt)

        y = getattr(obs, y_plot)
        yerr = obs.magErr
        x = getattr(obs, x_plot)

        #         y = abs_mag
        sort_mask = np.argsort(x)
        x = x[sort_mask]
        y = y[sort_mask]
        yerr = obs.magErr[sort_mask]
        # #         x_mask = (x>=time_min) & (x<time_max)
        x_mask = (x >= time_min) & (x <= _time_max)
        x = x[x_mask]
        y = y[x_mask]
        yerr = yerr[x_mask]
        # #         print(len(x))

        ax1.errorbar(x, y, yerr, fmt="o", label=filt)

        tck = splrep(x, y, w=1.0 / yerr)
        splines.append(tck)

        xnew = np.linspace(time_min, _time_max, 1000)
        ax1.plot(xnew, BSpline(*tck)(xnew), "-", label="weighted smooth", c="k", zorder=3)

ax1.set_xlabel(x_plot)
ax1.set_ylabel(y_plot)
ax1.legend()
ax1.invert_yaxis()

plt.show()

# test colour finding functions

In [ ]:
# # set number of reference observations to use for colour estimate
# # N_mean = 5
# N_mean = 3
# # N_mean = 1
# # N_mean = None

# # define observation column names to be analysed for colour
# x_plot = "midPointMjdTai"
# y_plot = "AbsMag"
# # y_plot = "reduced_mag"
# yerr_plot = "magErr"
# filt_obs = "g"
# filt_ref = "r"
# colour = "{}-{}".format(filt_obs,filt_ref)
# colErr = "{}-{}Err".format(filt_obs,filt_ref)
# delta_t_col = "delta_t_{}".format(colour)

# # define new columns to be added to the observations dataframe
# new_obs_cols = [colour,colErr,delta_t_col]

# fig = plt.figure()
# gs = gridspec.GridSpec(1, 1)
# ax1 = plt.subplot(gs[0, 0])

# for i in range(len(t_app)-1):

#     if i!=3:
#         continue

#     time_min = t_app[i]
#     time_max = t_app[i+1]
#     print(time_min,time_max)

#     _df_obs_all = df_obs_all[(df_obs_all["midPointMjdTai"]>=time_min) & (df_obs_all["midPointMjdTai"]<time_max)]
#     print(i,len(_df_obs_all))
#     _time_max = np.amax(_df_obs_all["midPointMjdTai"])

#     # get the phase curve model and observations for each filter

#     # get the stored AdlerData parameters for this filter
#     ad_g = adler_cols.get_phase_parameters_in_filter(filt_obs,"HG12_Pen16")
#     # make the PhaseCurve object from AdlerData
#     pc_g = PhaseCurve().InitModelDict(ad_g.__dict__)

#     ad_r = adler_cols.get_phase_parameters_in_filter(filt_ref,"HG12_Pen16")
#     pc_r = PhaseCurve().InitModelDict(ad_r.__dict__)

#     df_obs = get_df_obs_filt(planetoid,filt_obs,x1=time_min,x2=_time_max, col_list = [y_plot,yerr_plot], pc_model = pc_g)
#     df_obs_ref = get_df_obs_filt(planetoid,filt_ref,x1=time_min,x2=_time_max, col_list = [y_plot,yerr_plot], pc_model = pc_r)

#     ax1.errorbar(df_obs[x_plot], df_obs[y_plot], df_obs[yerr_plot], fmt="o", label = filt_obs)
#     ax1.errorbar(df_obs_ref[x_plot], df_obs_ref[y_plot], df_obs_ref[yerr_plot], fmt="o", label = filt_ref)

#     # set up some new columns for storing colour parameters
#     for x in new_obs_cols:
#         df_obs[x] = np.nan

#     for i in range(len(df_obs)):

#         # select the values of the new observation in the test filter
#         x_obs = df_obs.iloc[i][x_plot]
#         y_obs = df_obs.iloc[i][y_plot]
#         yerr_obs = df_obs.iloc[i][yerr_plot]

#         # select observations in the reference filter from before the new obs
#         ref_mask = (df_obs_ref[x_plot]<x_obs)

#         # set the number of ref obs to use
#         if N_mean is None:
#             _N_mean = len(df_obs_ref[ref_mask]) # use all available ref obs
#         else:
#             _N_mean = N_mean

#         # select only the N ref obs for comparison
#         _df_obs_ref = df_obs_ref[ref_mask].iloc[-_N_mean:]
#         if len(_df_obs_ref)==0:
#             print("no reference observations")
#             continue

#         # determine reference observation values
#         y_ref = np.mean(_df_obs_ref[y_plot])
#         yerr_ref = np.std(_df_obs_ref[y_plot]) # TODO: propagate ref uncertainty properly

#         # determine the ref obs time range
#         x1_ref = np.array(_df_obs_ref[x_plot])[0]
#         x2_ref = np.array(_df_obs_ref[x_plot])[-1]

#         # calculate the obs - ref y value
#         df_obs.loc[i,colour] = y_obs - y_ref
#         # find the time between new obs and last ref obs
#         df_obs.loc[i,delta_t_col] = x_obs - x2_ref
#         # record uncertainty on colour value
#         df_obs.loc[i,colErr] = np.sqrt((yerr_obs**2.0) + (yerr_ref**2.0))

#         # plot some lines to show the colour and mean reference
#         ax1.vlines(x_obs,y_obs,y_ref, color = "k", ls = ":")
#         ax1.hlines(y_ref,x1_ref,x2_ref, color = "k", ls = "--")

#         # TODO:
#         # could also record phase angle diff and phase curve residual?
#         # need to test error case where there are no r filter obs yet

# ax1.set_xlabel(x_plot)
# ax1.set_ylabel(y_plot)
# ax1.legend()
# ax1.invert_yaxis()

# plt.show()

In [ ]:
# # find g - r diff of newest g observation to the mean of the previous N r observations
# # colour code by time diff

# x_plot = "midPointMjdTai"
# y_plot = colour
# y_plot_err = colErr
# c_plot = delta_t_col
# df_plot = df_obs

# fig = plt.figure()
# gs = gridspec.GridSpec(1, 1)
# ax1 = plt.subplot(gs[0, 0])


# s1 = ax1.scatter(df_plot[x_plot],df_plot[y_plot],c=df_plot[c_plot], zorder=3)
# cbar1 = plt.colorbar(s1)
# ax1.errorbar(df_plot[x_plot],df_plot[y_plot],df_plot[yerr_plot],fmt = ".", zorder = 1)

# # TODO: leave the most recent obs out of this mean?
# # loop through new obs and calculate current mean and std?
# obs_ref_mean = np.mean(df_plot[y_plot])
# obs_ref_median = np.median(df_plot[y_plot])
# obs_ref_std = np.std(df_plot[y_plot])
# print(obs_ref_median,obs_ref_std)

# ax1.axhline(obs_ref_mean, c="k")
# ax1.axhspan(obs_ref_mean - obs_ref_std, obs_ref_mean + obs_ref_std, zorder = 0, color = "k", alpha = 0.2)

# ax1.set_xlabel(x_plot)
# ax1.set_ylabel(y_plot)
# cbar1.set_label(c_plot)

# plt.show()

In [ ]:
# define colour function parameters

test_app_i = 3
# set number of reference observations to use for colour estimate
# N_ref = 5
# N_ref = 3
# N_ref = 1
# N_ref = None

# observation and filter field names
x_plot = "midPointMjdTai"
y_plot = "AbsMag"
y_plot = "reduced_mag"
yerr_plot = "magErr"
filt_obs = "g"
filt_ref = "r"
obsId_col = "diaSourceId"

# define colour field names
colour = "{}-{}".format(filt_obs, filt_ref)
colErr = "{}-{}Err".format(filt_obs, filt_ref)
delta_t_col = "delta_t_{}".format(colour)
y_ref_col = "{}_{}".format(y_plot, filt_ref)
x1_ref_col = "{}1_{}".format(x_plot, filt_ref)
x2_ref_col = "{}2_{}".format(x_plot, filt_ref)

for N_ref in [1, 3, 5]:
    fig = plt.figure()
    gs = gridspec.GridSpec(1, 1)
    ax1 = plt.subplot(gs[0, 0])

    col_dict_list = []
    for app_i in range(len(t_app) - 1):
        if app_i != test_app_i:
            continue

        time_min = t_app[app_i]
        time_max = t_app[app_i + 1]

        _df_obs_all = df_obs_all[
            (df_obs_all["midPointMjdTai"] >= time_min) & (df_obs_all["midPointMjdTai"] < time_max)
        ]
        print(app_i, len(_df_obs_all))
        _time_max = np.amax(_df_obs_all["midPointMjdTai"])
        print(time_min, _time_max)

        # get the phase curve model and observations for each filter
        # get the stored AdlerData parameters for this filter
        ad_g = adler_cols.get_phase_parameters_in_filter(filt_obs, "HG12_Pen16")
        # make the PhaseCurve object from AdlerData
        pc_g = PhaseCurve().InitModelDict(ad_g.__dict__)
        ad_r = adler_cols.get_phase_parameters_in_filter(filt_ref, "HG12_Pen16")
        pc_r = PhaseCurve().InitModelDict(ad_r.__dict__)
        df_obs = get_df_obs_filt(
            planetoid,
            filt_obs,
            x1=time_min,
            x2=_time_max,
            col_list=[obsId_col, y_plot, yerr_plot],
            pc_model=pc_g,
        )
        df_obs_ref = get_df_obs_filt(
            planetoid, filt_ref, x1=time_min, x2=_time_max, col_list=[y_plot, yerr_plot], pc_model=pc_r
        )

        ax1.errorbar(df_obs[x_plot], df_obs[y_plot], df_obs[yerr_plot], fmt="o", label=filt_obs)
        ax1.errorbar(df_obs_ref[x_plot], df_obs_ref[y_plot], df_obs_ref[yerr_plot], fmt="o", label=filt_ref)

        # simulate stepping through each filt_obs observation
        x1 = time_min
        for xi in range(len(df_obs)):
            x2 = df_obs.iloc[xi][x_plot]
            print(xi, x1, x2)

            # do the colour finding function here
            col_dict = col_obs_ref(
                planetoid,
                adler_cols,
                filt_obs=filt_obs,
                filt_ref=filt_ref,
                N_ref=N_ref,
                x_col=x_plot,
                y_col=y_plot,
                yerr_col=yerr_plot,
                x1=x1,
                x2=x2,
            )
            col_dict_list.append(col_dict)

            # plot some lines to show the colour and mean reference
            ax1.vlines(
                df_obs.iloc[xi][x_plot], df_obs.iloc[xi][y_plot], col_dict[y_ref_col], color="k", ls=":"
            )
            ax1.hlines(col_dict[y_ref_col], col_dict[x1_ref_col], col_dict[x2_ref_col], color="k", ls="--")

    df_col = pd.DataFrame(col_dict_list)
    print(list(df_col))
    print(list(df_obs))
    #     df_col = pd.concat([df_obs,df_col], axis =1)
    df_col = df_col.merge(df_obs[[x for x in list(df_obs) if x != x_plot]], on=obsId_col)
    df_col["diaSourceId"] = df_col["diaSourceId"].astype(int)

    ax1.set_xlabel(x_plot)
    ax1.set_ylabel(y_plot)
    ax1.legend()
    ax1.invert_yaxis()

    plt.show()

    df_col_fname = "df_{}_{}_{}_app_{}_N_ref_{}.csv".format(ssoid, filt_obs, filt_ref, test_app_i, N_ref)
    df_col.to_csv(df_col_fname)

In [ ]:
df_obs

In [ ]:
col_dict

In [ ]:
df_col

In [ ]:
# df_col_fname = "df_{}_{}_{}_app_{}_N_ref_{}.csv".format(ssoid,filt_obs,filt_ref,test_app_i,N_ref)
# df_col_fname

In [ ]:
# df_col.to_csv(df_col_fname)

In [ ]:
# find filt_obs - filt_ref of newest filt_obs observation to the mean of the previous N_ref filt_ref observations
# colour code by time diff between obs and most recent obs_ref

x_plot = "midPointMjdTai"
y_plot = colour
y_plot_err = colErr
c_plot = delta_t_col
df_plot = df_col

fig = plt.figure()
gs = gridspec.GridSpec(1, 1)
ax1 = plt.subplot(gs[0, 0])


s1 = ax1.scatter(df_plot[x_plot], df_plot[y_plot], c=df_plot[c_plot], zorder=3)
cbar1 = plt.colorbar(s1)
ax1.errorbar(df_plot[x_plot], df_plot[y_plot], df_plot[yerr_plot], fmt=".", zorder=1)

# TODO: leave the most recent obs out of this mean?
# loop through new obs and calculate current mean and std?
obs_ref_mean = np.mean(df_plot[y_plot])
obs_ref_median = np.median(df_plot[y_plot])
obs_ref_std = np.std(df_plot[y_plot])
print(obs_ref_median, obs_ref_std)

ax1.axhline(obs_ref_mean, c="k")
ax1.axhspan(obs_ref_mean - obs_ref_std, obs_ref_mean + obs_ref_std, zorder=0, color="k", alpha=0.2)

ax1.set_xlabel(x_plot)
ax1.set_ylabel(y_plot)
cbar1.set_label(c_plot)

plt.show()

In [ ]:
# The colours can then be run through the previously written outlier detection functions

In [ ]:
planetoid.ssObjectId